In [3]:
#Part 1 Setup Code 
#begin by importing packages needed
import pandas as pd
import os
import numpy as np
from os import sep

In [4]:
#set current working directory path
cwd = os.getcwd()

In [7]:
#set up folders 
if not os.path.exists('stats_summative'):
    os.makedirs('stats_summative')

if not os.path.exists(f'stats_summative{sep}data'):
    os.makedirs(f'stats_summative{sep}data')

In [42]:
#download data from the Internet; unzip any that is currently in gz form
import gzip
import urllib
import requests

def download_data(url, name):
    '''
    Given a URL and a string, this function downloads data from the Internet and writes it into a file entitled name.
    '''
    r = requests.get(url)
    with open(f'{cwd}{sep}stats_summative{sep}data{sep}' + name, 'wb') as f:
         f.write(r.content)

def dataframe_upload(url, name, gz=False):
    ''' 
    Given a URL and string name, this function downloads data from the Internet, unzips it to a file entitled 'name' if it's a
    gzip file (when gzip=True), and returns a dataframe corresponding to that file
    '''
    r = requests.get(url)
    with open(f'{cwd}{sep}stats_summative{sep}data{sep}' + name, 'wb') as f:
         f.write(r.content)
    if gz == True:
        with gzip.open(f'{cwd}{sep}stats_summative{sep}data{sep}' + name) as f:
            df = pd.read_csv(f)
    else:
        df = pd.read_csv(f'{cwd}{sep}stats_summative{sep}data{sep}' + name)
    return df


In [171]:
# #use API to access data from NYC 311
# from sodapy import Socrata
# keyFile = open(f'{cwd}{sep}stats_summative{sep}apikey.txt', 'r')
# token = keyFile.readline() #api token imported from txt file

#uses Socrata from sodapy. as a public dataset, no auth is needed -- only an API token to eliminate throttling limits
# client = Socrata("data.cityofnewyork.us", token)
# results = client.get("erm2-nwe9", limit = 1000000, select= 'unique_key, created_date, complaint_type, descriptor, incident_address, latitude, longitude', \
#                     where = 'latitude IS NOT NULL AND (date_extract_y(created_date) = 2016 or date_extract_y(created_date) = 2017) \
#                     AND (complaint_type = "Noise - Residential" OR complaint_type = "Noise - Street/Sidewalk" OR \
#                     complaint_type = "Illegal Parking" OR complaint_type = "Rodent" OR complaint_type = "Illegal Fireworks" \
#                     OR complaint_type = "Urinating in Public" OR complaint_type = "Disorderly Youth" OR complaint_type = "Traffic" \
#                     OR complaint_type = "Drinking")')
# nyc_311 = pd.DataFrame.from_records(results)

# # download data where lat/long is null to investigate potential discrepancies 
nyc_null = results = client.get("erm2-nwe9", limit = 10000, select= 'unique_key, created_date, complaint_type, descriptor, incident_address, latitude, longitude', \
                    where = '(latitude IS NULL OR longitude IS NULL) AND (date_extract_y(created_date) = 2016 or date_extract_y(created_date) = 2017) \
                    AND (complaint_type = "Noise - Residential" OR complaint_type = "Noise - Street/Sidewalk" OR \
                    complaint_type = "Illegal Parking" OR complaint_type = "Rodent" OR complaint_type = "Illegal Fireworks" \
                    OR complaint_type = "Urinating in Public" OR complaint_type = "Disorderly Youth" OR complaint_type = "Traffic" \
                    OR complaint_type = "Drinking")')
nyc_null = pd.DataFrame.from_records(nyc_null)
print(nyc_null.complaint_type.value_counts(), "\n \n")
print(nyc_311.complaint_type.value_counts())

Illegal Parking            3221
Noise - Residential        1396
Noise - Street/Sidewalk    1249
Rodent                      395
Traffic                      98
Drinking                     15
Urinating in Public          10
Illegal Fireworks             8
Disorderly Youth              4
Name: complaint_type, dtype: int64 
 

Noise - Residential        450737
Illegal Parking            265498
Noise - Street/Sidewalk    133066
Rodent                      66119
Traffic                     11125
Drinking                     3022
Urinating in Public           986
Illegal Fireworks             448
Disorderly Youth              389
Name: complaint_type, dtype: int64


In [222]:
#download other cities' data
# boston_311_18 = dataframe_upload("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv", \
#                                 'boston_311_18')
# boston_311_19 = dataframe_upload("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv", \
#                                 'boston_311_19')
# boston_311_20 = dataframe_upload("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpbi76tky0.csv", \
#                                 'boston_311_20')
#boston_311_raw = boston_311_18.append(boston_311_19.append(boston_311_20))
boston_311 = boston_311_raw[(boston_311_raw['reason'] == "Generic Noise Disturbance") |  (boston_311_raw['reason'] == "Parking Complaints")]

In [223]:
#download Airbnb data
#NYC: January 2016 to December 2017
nyc_dates = ["2016-01-01", "2016-02-02", "2016-04-03", "2016-05-02", "2016-06-02", "2016-07-02", "2016-08-02", "2016-09-02", "2016-10-01", "2016-11-02", \
            "2016-12-03", "2017-01-01", "2017-02-02", "2017-03-02", "2017-04-02", "2017-05-02", "2017-06-02", "2017-07-02", "2017-08-02", "2017-09-02", \
            "2017-10-02", "2017-11-01", "2017-12-02"]
test_dates = ["2016-01-01", "2016-02-02"]
for date in test_dates:
    df = dataframe
    
    
    
    
    

,owner_department
0,Department of Housing
1,DWM - Department of Water Management
2,CDOT - Department of Transportation
3,Animal Care and Control
4,311 City Services
5,City Clerk's Office
6,BACP - Business Affairs and Consumer Protection
7,Fire
8,Department of Planning and Development
9,Streets and Sanitation


In [ ]:
print("halla")